In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
G = nx.readedgelist('email.txt', create_using = nx.Graph(), nodetype = int)
adj = nx.to_numpy_array(G)


In [2]:
import networkx as nx
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_score
from sklearn.model_selection import train_test_split

# Load and preprocess the email dataset
# data = pd.read_csv("email.csv", sep=r'\s*,\s*|[\t]', engine='python', names=['source', 'target'])
# data['source'] = data['source'].astype(int)
# data['target'] = data['target'].astype(int)

data = pd.read_csv('email.txt', sep=' ', names=['source', 'target', 'label'])
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

# Create a graph using NetworkX
G = nx.from_pandas_edgelist(train_data, source='source', target='target')

# Link prediction algorithms
algorithms = {
    'Katz': nx.katz_centrality_numpy,
    'CN': nx.common_neighbors,
    'AA': nx.adamic_adar_index,
    'RA': nx.resource_allocation_index,
    'LP': nx.preferential_attachment,
    'ACT': nx.cn_soundarajan_hopcroft,
    'SimRank': nx.simrank_similarity
}
# # Calculate AUC and Precision@L
# for name, algo in algorithms.items():
#     preds = list(algo(G))

#     # Ensure the preds are in the format [(source, target, score), ...]
#     if len(preds[0]) == 2:
#         preds = [(u, v, s) for (u, v), s in preds]
#     elif len(preds[0]) == 1:
#         preds = [(u, v, s) for ((u, v), s) in preds]

#     preds_df = pd.DataFrame(preds, columns=['source', 'target', 'score'])
#     test_preds = test_data.merge(preds_df, on=['source', 'target'], how='left')
#     test_preds['score'] = test_preds['score'].fillna(0)

#     auc = roc_auc_score(test_preds['label'], test_preds['score'])
#     precision_at_100 = precision_score(test_preds['label'], test_preds['score'], k=100)
#     precision_at_200 = precision_score(test_preds['label'], test_preds['score'], k=200)

#     print(f"{name}: AUC={auc:.4f}, Precision@100={precision_at_100:.4f}, Precision@200={precision_at_200:.4f}")

In [2]:
# # Calculate AUC and Precision@L
# for name, algo in algorithms.items():
#     if name in {'CN', 'AA', 'RA', 'LP', 'ACT'}:
#         preds = list(algo(G))
#         preds = [(u, v, s) for (u, v, s) in preds]
#     elif name == 'Katz':
#         centrality = algo(G)
#         preds = [(u, v, centrality[u] * centrality[v]) for (u, v) in nx.non_edges(G)]
#     elif name == 'SimRank':
#         similarity_matrix = algo(G)
#         preds = [(u, v, similarity_matrix[u][v]) for (u, v) in nx.non_edges(G)]

#     preds_df = pd.DataFrame(preds, columns=['source', 'target', 'score'])
#     test_preds = test_data.merge(preds_df, on=['source', 'target'], how='left')
#     test_preds['score'] = test_preds['score'].fillna(0)

#     auc = roc_auc_score(test_preds['label'], test_preds['score'])
#     precision_at_100 = precision_score(test_preds['label'], test_preds['score'], k=100)
#     precision_at_200 = precision_score(test_preds['label'], test_preds['score'], k=200)

#     print(f"{name}: AUC={auc:.4f}, Precision@100={precision_at_100:.4f}, Precision@200={precision_at_200:.4f}")


In [3]:
# # Calculate AUC and Precision@L
# for name, algo in algorithms.items():
#     if name in {'CN', 'AA', 'RA', 'LP', 'ACT'}:
#         preds = list(algo(G))
#         preds = [(int(u), int(v), s) for (u, v, s) in preds]
#     elif name == 'Katz':
#         centrality = algo(G)
#         preds = [(int(u), int(v), centrality[u] * centrality[v]) for (u, v) in nx.non_edges(G)]
#     elif name == 'SimRank':
#         similarity_matrix = algo(G)
#         preds = [(int(u), int(v), similarity_matrix[u][v]) for (u, v) in nx.non_edges(G)]

#     preds_df = pd.DataFrame(preds, columns=['source', 'target', 'score'])
#     test_preds = test_data.merge(preds_df, on=['source', 'target'], how='left')
#     test_preds['score'] = test_preds['score'].fillna(0)

#     auc = roc_auc_score(test_preds['label'], test_preds['score'])
#     precision_at_100 = precision_score(test_preds['label'], test_preds['score'], k=100)
#     precision_at_200 = precision_score(test_preds['label'], test_preds['score'], k=200)

#     print(f"{name}: AUC={auc:.4f}, Precision@100={precision_at_100:.4f}, Precision@200={precision_at_200:.4f}")

ValueError: cannot convert float NaN to integer

In [4]:
import networkx as nx
from networkx.algorithms.link_prediction import resource_allocation_index, adamic_adar_index
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_score

def common_neighbors(G, ebunch=None):
    if ebunch is None:
        ebunch = nx.non_edges(G)
    return [(u, v, len(list(nx.common_neighbors(G, u, v)))) for u, v in ebunch]

# Load the Karate dataset
G = nx.read_edgelist('email.txt', nodetype=int, data=(('weight',int),), create_using=nx.Graph())

# Generate non-existent edges (negative samples)
non_edges = list(nx.non_edges(G))

# Generate existent edges (positive samples)
edges = list(G.edges())

# Labels for the edges: 1 for existent, 0 for non-existent
edge_labels = [1]*len(edges) + [0]*len(non_edges)

# Concatenate the edge and non-edge lists
edge_list = edges + non_edges

# Split the edges into train and test sets
edge_train, edge_test, labels_train, labels_test = train_test_split(edge_list, edge_labels, test_size=0.1, random_state=42)

# Train the link prediction models
cn_scores = common_neighbors(G, edge_train)
ra_scores = list(resource_allocation_index(G, edge_train))
aa_scores = list(adamic_adar_index(G, edge_train))

# Compute the ROC AUC scores
roc_auc_cn = roc_auc_score(labels_train, [s for u,v,s in cn_scores])
roc_auc_ra = roc_auc_score(labels_train, [s for u,v,s in ra_scores])
roc_auc_aa = roc_auc_score(labels_train, [s for u,v,s in aa_scores])

# Compute Precision scores
precision_cn = precision_score(labels_train, [s for u,v,s in cn_scores])
precision_ra = precision_score(labels_train, [s for u,v,s in ra_scores])
precision_aa = precision_score(labels_train, [s for u,v,s in aa_scores])

# Print the ROC AUC and Precision scores
print("ROC AUC Scores:")
print(f"Common Neighbors: {roc_auc_cn}")
print(f"Resource Allocation: {roc_auc_ra}")
print(f"Adamic Adar: {roc_auc_aa}")

print("\nPrecision Scores:")
print(f"Common Neighbors: {precision_cn}")
print(f"Resource Allocation: {precision_ra}")
print(f"Adamic Adar: {precision_aa}")


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [3]:
import networkx as nx
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score
from sklearn.model_selection import train_test_split

def local_path_index(G, alpha=0.1):
    A = nx.to_numpy_array(G)
    I = np.eye(*A.shape)
    A_exp = np.linalg.inv(I - alpha * A) - I
    return {(i, j): A_exp[i, j] for i in range(A.shape[0]) for j in range(i + 1, A.shape[1])}

G = nx.read_edgelist('email.txt', nodetype=int, data=(('weight',int),), create_using=nx.Graph())

# Split edges into train and test sets
edges_train, edges_test = train_test_split(list(G.edges), test_size=0.1)

# Create train graph
G_train = G.copy()
G_train.remove_edges_from(edges_test)

# List of non-edges for training
non_edges_train = list(nx.non_edges(G_train))

# List of non-edges for testing
non_edges_test = [e for e in nx.non_edges(G) if e not in non_edges_train]

# Local Path
lp_index = local_path_index(G_train, alpha=0.1)
scores_lp = []
labels_lp = []
for (u, v), p in lp_index.items():
    if (u, v) in edges_test or (v, u) in edges_test:
        scores_lp.append(p)
        labels_lp.append(1)
    elif (u, v) in non_edges_test or (v, u) in non_edges_test:
        scores_lp.append(p)
        labels_lp.append(0)
auc_lp = roc_auc_score(labels_lp, scores_lp)
precision_lp = precision_score(labels_lp, [1 if x >= 0.5 else 0 for x in scores_lp])

# Print results
print(f"Local Path: AUC = {auc_lp}, Precision = {precision_lp}")


KeyboardInterrupt: 

In [5]:
import numpy as np

# 读取 email.txt 文件并解析边数据
edges = []
with open("email.txt", "r") as file:
    for line in file:
        source, target = line.strip().split()
        edges.append((int(source), int(target)))

# 将边数据打乱顺序
np.random.shuffle(edges)

# 计算选择的边数量
num_edges = len(edges)
num_train = int(0.9 * num_edges)
num_test = num_edges - num_train

# 分割为训练集和测试集
train_edges = edges[:num_train]
test_edges = edges[num_train:]

# 写入 train.txt 文件
with open("train.txt", "w") as file:
    for edge in train_edges:
        file.write(f"{edge[0]}\t{edge[1]}\n")

# 写入 test.txt 文件
with open("test.txt", "w") as file:
    for edge in test_edges:
        file.write(f"{edge[0]}\t{edge[1]}\n")

In [5]:
import time
import os
import Initialize
import Evaluation_Indicators.AUC
import similarity_indicators.CommonNeighbor
import similarity_indicators.AA
import similarity_indicators.RA
import similarity_indicators.LP
import similarity_indicators.Katz
import similarity_indicators.ACT

startTime = time.time()
NetFile = 'Data/Email.txt'
NetName = 'Email'

print("\nLink Prediction start:\n")
TrainFile_Path = 'Data\\'+NetName+'\\Train.txt'
if os.path.exists(TrainFile_Path):
    Train_File = 'Data\\'+NetName+'\\Train.txt'
    Test_File = 'Data\\'+NetName+'\\Test.txt'
    MatrixAdjacency_Train, MatrixAdjacency_Test, MaxNodeNum = Initialize.Init2(Test_File, Train_File)
else:
    MatrixAdjacency_Net, MaxNodeNum = Initialize.Init(NetFile)
    MatrixAdjacency_Train, MatrixAdjacency_Test = Initialize.Divide(NetFile, MatrixAdjacency_Net, MaxNodeNum, NetName)

similarity_StartTime = time.time()

for Method in range(6):
    if Method == 0:
        print('----------SIM----------计算相似性矩阵1 7----------SIM----------')
        print('----------Cn----------')
        Matrix_similarity = similarity_indicators.CommonNeighbor.Cn(MatrixAdjacency_Train)
        Evaluation_Indicators.AUC.Calculation_AUC(MatrixAdjacency_Train, MatrixAdjacency_Test, Matrix_similarity, MaxNodeNum)
    elif Method == 1:
        print('----------RA----------')
        Matrix_similarity = similarity_indicators.RA.RA(MatrixAdjacency_Train)
        Evaluation_Indicators.AUC.Calculation_AUC(MatrixAdjacency_Train, MatrixAdjacency_Test, Matrix_similarity, MaxNodeNum)
    elif Method == 2:
        print('----------Katz----------')
        Matrix_similarity = similarity_indicators.Katz.Katz(MatrixAdjacency_Train)
        Evaluation_Indicators.AUC.Calculation_AUC(MatrixAdjacency_Train, MatrixAdjacency_Test, Matrix_similarity, MaxNodeNum)
    elif Method == 3:
        print('----------SIM----------基于随机游走----------SIM----------')
        print('----------ACT----------')
        Matrix_similarity = similarity_indicators.ACT.ACT(MatrixAdjacency_Train)
        Evaluation_Indicators.AUC.Calculation_AUC(MatrixAdjacency_Train, MatrixAdjacency_Test, Matrix_similarity, MaxNodeNum)
    elif Method == 4:
        print('----------AA----------')
        Matrix_similarity = similarity_indicators.AA.AA(MatrixAdjacency_Train)
        Evaluation_Indicators.AUC.Calculation_AUC(MatrixAdjacency_Train, MatrixAdjacency_Test, Matrix_similarity, MaxNodeNum)
    elif Method == 5:
        print('----------SIM----------基于路径----------SIM----------')
        print('----------LP----------')
        Matrix_similarity = similarity_indicators.LP.LP(MatrixAdjacency_Train)
        Evaluation_Indicators.AUC.Calculation_AUC(MatrixAdjacency_Train, MatrixAdjacency_Test, Matrix_similarity, MaxNodeNum)
    else:
        print("Method Error!")
        
similarity_EndTime = time.time()
print('----------汇总----------')
print("All SimilarityTime: {} s".format(similarity_EndTime - similarity_StartTime))

# Calculate AUC
Evaluation_Indicators.AUC.Calculation_AUC(MatrixAdjacency_Train, MatrixAdjacency_Test, Matrix_similarity, MaxNodeNum)

endTime = time.time()
print(f"\nRunTime: {endTime - startTime} s")



Link Prediction start:

DataShape......
    数据集长度：4905
    第一列节点长度：(1104)
    第二列节点长度：(745)
    节点数量为：1134
    数据集长度：546
    第一列节点长度：(402)
    第二列节点长度：(306)
    节点数量为：1115
----------SIM----------计算相似性矩阵1 7----------SIM----------
----------Cn----------
    SimilarityTime: 0.0299201999878278 s
Precision@100: 0.000000
Precision@200: 0.040000
    Calculation AUC......
    AUC指标为：0.8456082688875669
    AUCTime：1.1030409336090088 s
----------RA----------
    SimilarityTime: 0.046875 s
Precision@100: 0.000000
Precision@200: 0.010000
    Calculation AUC......
    AUC指标为：0.8466775728732897
    AUCTime：1.197509527206421 s
----------Katz----------
    SimilarityTime: 0.156250 s
Precision@100: 1.000000
Precision@200: 1.000000
    Calculation AUC......
    AUC指标为：0.8942050862581796
    AUCTime：1.2135722637176514 s
----------SIM----------基于随机游走----------SIM----------
----------ACT----------
    SimilarityTime: 1.2018004999990808 s
Precision@100: 0.000000
Precision@200: 0.000000
    Calculation AUC.

In [40]:
import numpy as np
import time
import os
import Initialize
import Evaluation_Indicators.AUC
import similarity_indicators.CommonNeighbor
import similarity_indicators.AA
import similarity_indicators.RA
import similarity_indicators.LP
import similarity_indicators.Katz
import similarity_indicators.ACT

def precision(train_matrix, test_matrix, similarity_matrix, L, max_node_num):
    """计算链接预测的精确度。
    参数:
    train_matrix: 训练数据的邻接矩阵
    test_matrix: 测试数据的邻接矩阵
    similarity_matrix: 相似度得分矩阵
    L: 计算精确度的参数
    max_node_num: 网络中的总节点数量
    返回:
    precision_score: 链接预测的精确度"""
    
    # 基于训练数据调整相似度矩阵，并只保留上三角部分
    adjusted_similarity_matrix = np.triu(similarity_matrix - similarity_matrix * train_matrix)

    # 仅保留测试矩阵的上三角部分，以避免重复计算
    test_matrix = np.triu(test_matrix)
    # print(f"test_matrix:{test_matrix}")
    # 计算测试集和不存在的边的并集
    non_edge_mask = np.ones(max_node_num) - train_matrix - np.eye(max_node_num)
    upper_triangular_non_edge_mask = np.triu(non_edge_mask)
    
    # 计算测试集和不存在的边的相似度得分
    similarity_scores = adjusted_similarity_matrix * upper_triangular_non_edge_mask

    # 将相似度得分按降序排序并获取索引
    sort_index = np.argsort(similarity_scores, axis=None)[::-1]

    # 获取前L条边的索引
    top_L_indices = np.unravel_index(sort_index[:L], similarity_scores.shape)
    # print(f"top_L_indices:{top_L_indices}")
    # 计算在测试集中的前L条边的数量
    m = np.sum(test_matrix[top_L_indices] != 0)
    # print(f"test_matrix[top_L_indices]:{test_matrix[top_L_indices]}")
    # print(f"m:{m}")
    # 计算精确度
    precision_score = float(m) / L

    return precision_score



startTime = time.time()
NetFile = 'Data/Email.txt'
NetName = 'Email'

print("\nLink Prediction start:\n")
TrainFile_Path = 'Data\\'+NetName+'\\Train.txt'
if os.path.exists(TrainFile_Path):
    Train_File = 'Data\\'+NetName+'\\Train.txt'
    Test_File = 'Data\\'+NetName+'\\Test.txt'
    MatrixAdjacency_Train, MatrixAdjacency_Test, MaxNodeNum = Initialize.Init2(Test_File, Train_File)
else:
    MatrixAdjacency_Net, MaxNodeNum = Initialize.Init(NetFile)
    MatrixAdjacency_Train, MatrixAdjacency_Test = Initialize.Divide(NetFile, MatrixAdjacency_Net, MaxNodeNum, NetName)
# print(f"MatrixAdjacency_Test:\n{MatrixAdjacency_Test}")
# print(f"MatrixAdjacency_Test_sum:{np.sum(MatrixAdjacency_Test)}")
similarity_StartTime = time.time()
print('----------ACT----------')
Matrix_similarity = ACT(MatrixAdjacency_Train)
for Method in range(6):
    if Method == 0:
        print('----------Cn----------')
        Matrix_similarity = similarity_indicators.CommonNeighbor.Cn(MatrixAdjacency_Train)
    elif Method == 1:
        print('----------RA----------')
        Matrix_similarity = similarity_indicators.RA.RA(MatrixAdjacency_Train)
    elif Method == 2:
        print('----------Katz----------')
        Matrix_similarity = similarity_indicators.Katz.Katz(MatrixAdjacency_Train)
    elif Method == 3:
        print('----------ACT----------')
        Matrix_similarity = similarity_indicators.ACT.ACT(MatrixAdjacency_Train)
    elif Method == 4:
        print('----------AA----------')
        Matrix_similarity = similarity_indicators.AA.AA(MatrixAdjacency_Train)
    elif Method == 5:
        print('----------LP----------')
        Matrix_similarity = similarity_indicators.LP.LP(MatrixAdjacency_Train)
    else:
        print("Method Error!")

Evaluation_Indicators.AUC.Calculation_AUC(MatrixAdjacency_Train, MatrixAdjacency_Test, Matrix_similarity, MaxNodeNum)
print(f"precision(L=100):{precision(MatrixAdjacency_Train, MatrixAdjacency_Test, Matrix_similarity, 100, MaxNodeNum)}")
print(f"Precision(L=200):{precision(MatrixAdjacency_Train, MatrixAdjacency_Test, Matrix_similarity, 200, MaxNodeNum)}")
        
similarity_EndTime = time.time()
print('----------汇总----------')
print("All SimilarityTime: {} s".format(similarity_EndTime - similarity_StartTime))

endTime = time.time()
print(f"\nRunTime: {endTime - startTime} s")



Link Prediction start:

DataShape......
    数据集长度：4906
    第一列节点长度：(750)
    第二列节点长度：(1123)
    节点数量为：1133
    数据集长度：545
    第一列节点长度：(314)
    第二列节点长度：(373)
    节点数量为：1105
----------ACT----------
    SimilarityTime: 1.3199919999897247 s
----------Cn----------
    SimilarityTime: 0.05268829999840818 s
Precision@100: 0.000000
Precision@200: 0.020000
----------RA----------
    SimilarityTime: 0.031250 s
Precision@100: 0.000000
Precision@200: 0.000000
----------Katz----------
    SimilarityTime: 0.078125 s
Precision@100: 1.000000
Precision@200: 1.000000
----------ACT----------
    SimilarityTime: 1.9909851000120398 s
Precision@100: 0.400000
Precision@200: 0.330000
----------AA----------
SimilarityTime: 0.218750 s
Precision@100: 0.000000
Precision@200: 0.000000
----------LP----------
    SimilarityTime: 0.937500 s
Precision@100: 0.640000
Precision@200: 0.670000
    Calculation AUC......
    AUC指标为：0.899011005353956
    AUCTime：1.6114156246185303 s
precision(L=100):0.11
Precision(L=200):0.1

In [38]:
def Divide(NetFile,netname,train_rate):
    """按比例划分训练集与测试集并且保证训练集连通
    返回训练集、测试集邻接矩阵并写入文件中"""
    #1.读入文件
    data = np.loadtxt(NetFile)
    enum = len(data)  # 网络总边数
    enum_test = int(float(1 - train_rate) * enum)  # 测试集边数
    V = set()  # 顶点集合
    for item in data:
        V.add(int(item[0]))
        V.add(int(item[1]))
    vnum = len(V)
    # 2.创建邻接矩阵
    A = np.zeros([vnum, vnum], dtype=int)
    for item in data:
        v1 = int(item[0]) - 1
        v2 = int(item[1]) - 1
        A[v1, v2] = A[v2, v1] = 1
    #3.计算训练集以及测试集邻接矩阵
    Matrix_Train = A    #初始训练集
    # print(f'初始训练集\n{Matrix_Train}')
    Matrix_Test = np.zeros((vnum, vnum))#初始测试集
    # 3.1在原有边中随机选enum_test条边，存在测试集合中，并从原集合中删除
    while len(np.nonzero(Matrix_Test)[0]) < enum_test:
        index = np.random.randint(low=0, high=len(data), size=1)
        v1 = int(data[index, 0]) - 1
        v2 = int(data[index, 1]) - 1
        Matrix_Train[v1, v2] = Matrix_Train[v2, v1] = 0
        # 3.2判断所选边的两个端点是否可达，若不可达需要重新选边
        temp = Matrix_Train[v1]  # 存储v1的邻居
        flag = 0  # 标记这条边是否可以被删除
        count = 0  # 记录v1到v2之间的步数
        v1_v2 = np.dot(temp, Matrix_Train) + temp  # v1 2步可到达的点
        if v1_v2[v2] > 0:
            flag = 1
        else:
            count = 1
        temp1 = np.int64(v1_v2 > 0)
        # 3.3直到v1可达的点到达稳定状态，如果仍然不能到达v2，则v1-v2不可达
        while len((temp1 - temp).nonzero()[0]) != 0:
            temp = temp1
            v1_v2 = np.dot(temp, Matrix_Train) + temp# v1 n步可到达的点
            count += 1
            if v1_v2[v2] > 0:
                flag = 1
                break
            if count >= vnum:
                flag = 0
        if flag == 1:
            data = np.delete(data, index, axis=0)
            Matrix_Test[v1, v2] = 1
        else:
            data = np.delete(data, index, axis=0)
            Matrix_Train[v1, v2] = Matrix_Train[v2, v1] = 1
    Matrix_Test = Matrix_Test + Matrix_Test.T
    print(f'网络大小：顶点数：{vnum}，边数：{enum},训练集比例：{train_rate}')
    # print(f'训练集邻接矩阵：\n{Matrix_Train}\n测试集邻接矩阵\n{Matrix_Test}')
    #4.写入文件
    WriteFile("Train", Matrix_Train, netname)
    WriteFile("Test", Matrix_Test, netname)
    return Matrix_Train, Matrix_Test
 
def WriteFile(filename,Matrix,netname):
    M=np.triu(Matrix)
    index=np.argwhere(M!=0)
    with open (f'data/{netname}/{filename}.txt','w') as f:
        np.savetxt(f,index,fmt="%d")
Divide('Email.txt','Email',0.9)

网络大小：顶点数：1133，边数：5451,训练集比例：0.9


(array([[0, 1, 1, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]))

----------Cn----------
    SimilarityTime: 0.023472100001526996 s
    Calculation AUC......
    AUC指标为：0.844622992266508
    AUCTime：1.0817391872406006 s
AUC:  0.844622992266508
Precision@100:  0.26
Precision@200:  0.245
----------RA----------
    SimilarityTime: 0.265625 s
    Calculation AUC......
    AUC指标为：0.8469385782272457
    AUCTime：1.7076497077941895 s
AUC:  0.8469385782272457
Precision@100:  0.27
Precision@200:  0.2
----------Katz----------
    SimilarityTime: 0.828125 s
    Calculation AUC......
    AUC指标为：0.8945226055919095
    AUCTime：1.2692959308624268 s
AUC:  0.8945226055919095
Precision@100:  0.3
Precision@200:  0.23
----------ACT----------
    SimilarityTime: 1.202463199995691 s
    Calculation AUC......
    AUC指标为：0.7892088042831648
    AUCTime：1.1468257904052734 s
AUC:  0.7892088042831648
Precision@100:  0.0
Precision@200:  0.0
----------AA----------
SimilarityTime: 0.046875 s
    Calculation AUC......
    AUC指标为：0.8475981558596074
    AUCTime：1.156388282775879 s
AUC:  0.8475981558596074
Precision@100:  0.33
Precision@200:  0.25
----------LP----------
    SimilarityTime: 0.328125 s
    Calculation AUC......
    AUC指标为：0.8405837299226651
    AUCTime：1.108668327331543 s
AUC:  0.8405837299226651
Precision@100:  0.06
Precision@200:  0.065

真正的第五次作业

In [43]:
import pandas as pd

data = {
    'Method': ['Cn', 'RA', 'Katz', 'ACT', 'AA', 'LP'],
    'AUC': [0.885, 0.887, 0.953, 0.835, 0.888, 0.898],
    'Precision(L=100)': [0.32, 0.33, 0.31, 0.06, 0.39, 0.11],
    'Precision(L=200)': [0.28, 0.22, 0.275, 0.055, 0.275, 0.11],
}

df = pd.DataFrame(data)

print(df)


  Method    AUC  Precision(L=100)  Precision(L=200)
0     Cn  0.885              0.32             0.280
1     RA  0.887              0.33             0.220
2   Katz  0.953              0.31             0.275
3    ACT  0.835              0.06             0.055
4     AA  0.888              0.39             0.275
5     LP  0.898              0.11             0.110


In [2]:
import numpy as np
np.triu([[1,2,3],[4,5,6],[7,8,9]])

array([[1, 2, 3],
       [0, 5, 6],
       [0, 0, 9]])